# Intro

[Codebook](http://start.umd.edu/gtd/downloads/Codebook.pdf)

## For prediction of conflictual zones intensities

0. Estimate global risque metric (function of #death, #injured, ...)

1. Look at points (zones) in space (Middle east) over all years.
2. Graph too dense, merge data to smaller clusters (look for technique to do it efficiently)
3. Plot evolution over time or current situatation
4. Fit model prediction (step = month ?)

[Stationary signal processing on graphs](https://arxiv.org/pdf/1601.02522.pdf)

[Predicting the evolution of stationary graph signals](https://arxiv.org/pdf/1607.03313.pdf)

[A GRAPH DOWNSAMPLING TECHNIQUE BASED On GRAPH FOURIER TRANSFORM](https://arxiv.org/pdf/1612.07542.pdf)

In [2]:
import pandas as pd
import os

In [3]:
data_path = 'data'
gtd_path = os.path.join(data_path, 'globalterrorismdb_0617dist.csv')

In [17]:
df = pd.read_csv(gtd_path, nrows=2000, encoding='latin')
df.head()

,eventid,iyear,imonth,iday,approxdate,extended,resolution,country,country_txt,region,...,addnotes,scite1,scite2,scite3,dbsource,INT_LOG,INT_IDEO,INT_MISC,INT_ANY,related
0,197000000001,1970,7,2,NaN,0,NaN,58,Dominican Republic,2,...,NaN,NaN,NaN,NaN,PGIS,0,0,0,0,NaN
1,197000000002,1970,0,0,NaN,0,NaN,130,Mexico,1,...,NaN,NaN,NaN,NaN,PGIS,0,1,1,1,NaN
2,197001000001,1970,1,0,NaN,0,NaN,160,Philippines,5,...,NaN,NaN,NaN,NaN,PGIS,-9,-9,1,1,NaN
3,197001000002,1970,1,0,NaN,0,NaN,78,Greece,8,...,NaN,NaN,NaN,NaN,PGIS,-9,-9,1,1,NaN
4,197001000003,1970,1,0,NaN,0,NaN,101,Japan,4,...,NaN,NaN,NaN,NaN,PGIS,-9,-9,1,1,NaN


In [20]:
import numpy as np

df.columns[np.sum(df.isnull(), axis=0) < 50]

Index(['eventid', 'iyear', 'imonth', 'iday', 'extended', 'country',
       'country_txt', 'region', 'region_txt', 'city', 'latitude', 'longitude',
       'specificity', 'vicinity', 'crit1', 'crit2', 'crit3', 'doubtterr',
       'multiple', 'success', 'suicide', 'attacktype1', 'attacktype1_txt',
       'targtype1', 'targtype1_txt', 'target1', 'natlty1', 'natlty1_txt',
       'gname', 'guncertain1', 'individual', 'weaptype1', 'weaptype1_txt',
       'property', 'ishostkid', 'ransom', 'dbsource', 'INT_LOG', 'INT_IDEO',
       'INT_MISC', 'INT_ANY'],
      dtype='object')

In [21]:
df['latitude']

0       18.456792
1       19.432608
2       15.478598
3       37.983773
4       33.580412
5       37.005105
6      -34.891151
7       37.805065
8       43.076592
9       43.072950
10      43.468500
11      39.740010
12      41.890520
13      42.331685
14      18.399712
15      52.516667
16            NaN
17      40.610069
18      18.379998
19      47.603560
20      40.116748
21     -34.891151
22      47.610594
23      47.655335
24      40.728075
25      14.624422
26      14.677301
27      10.502961
28      42.470310
29      33.606510
          ...    
1970    51.457935
1971   -22.908278
1972    54.576340
1973    54.994510
1974    54.467967
1975          NaN
1976    52.372068
1977    52.372068
1978   -34.603700
1979    44.407062
1980    45.463681
1981   -34.603700
1982    48.796696
1983    51.457935
1984   -34.603700
1985   -34.603700
1986   -34.603700
1987   -31.399301
1988    20.673343
1989    20.673343
1990    54.184008
1991   -32.951052
1992    42.817988
1993    54.597269
1994    33